# 毕业论文
## 前置数据处理：使用`pygrib`读取FNL
`Read FNL file Using pygrib`
---
*@author: Evan*\
*@date: 2023-10-07*

In [1]:
import pygrib
import numpy as np
import os

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../../src/')
from namelist import *

In [22]:
def read_grib2_files(years, month, day, hour, variable_names, dir):
    # Initialize an array to store data from each year
    yearly_message = {}
    yearly_data = {}

    for year in years:
        # Construct the file name based on the specified pattern
        file_name = f'fnl_{year:04d}{month:02d}{day:02d}_{hour:02d}_00.grib2'
        file_path = os.path.join(dir, file_name)

        # Open the GRIB2 file
        grb = pygrib.open(file_path)

        # Select the variable based on the given name
        messages = grb.select(name=variable_names)

        # Append the data to the yearly_data list
        yearly_message[year] = messages
        yearly_data[year] = [message.values for message in messages]
        # yearly_data.extend([message.values for message in messages])

        # Close the GRIB2 file
        grb.close()

    # Convert the list of arrays to a NumPy array for easier manipulation
    # yearly_data = np.array(yearly_data)

    return yearly_message, yearly_data

红箭头指向的变量在2014的FNL中没找到
![](https://picgo-evan.oss-cn-guangzhou.aliyuncs.com/img/202310072037924.png)

In [3]:
years = np.arange(2014,2016+1)
month = 6
day   = 16
hour  = 0

variables = ['Temperature','U component of wind','V component of wind',
             'Geopotential height','Relative humidity','Surface pressure',
             'Mean sea level pressure','Volumetric soil moisture content',
             'Sea ice area fraction','Land-sea mask',
             'Water equivalent of accumulated snow depth (deprecated)',
             'Pressure','Tropopause pressure',]



In [51]:
messages, data = read_grib2_files(years,month,day,hour,variables,batchdir)

In [15]:
messages[2016]

[1:U component of wind:m s**-1 (instant):regular_ll:planetaryBoundaryLayer:level 0:fcst time 0 hrs:from 201606160000,
 2:V component of wind:m s**-1 (instant):regular_ll:planetaryBoundaryLayer:level 0:fcst time 0 hrs:from 201606160000,
 5:Geopotential height:gpm (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 0 hrs:from 201606160000,
 6:Temperature:K (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 0 hrs:from 201606160000,
 7:Relative humidity:% (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 0 hrs:from 201606160000,
 8:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 0 hrs:from 201606160000,
 9:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 0 hrs:from 201606160000,
 11:Geopotential height:gpm (instant):regular_ll:isobaricInhPa:level 200 Pa:fcst time 0 hrs:from 201606160000,
 12:Temperature:K (instant):regular_ll:isobaricInhPa:level 200 Pa:fcst time 0 hrs:from 201606160000,
 13:Rela

In [38]:
data[2016][0].shape

(181, 360)

依据2014年元信息对后续年份做筛选

In [61]:
info = []
for msg in messages[2014]:
    if msg['name'] not in info:
        info.append(msg['name'])
info

['Mean sea level pressure',
 'Geopotential height',
 'Temperature',
 'U component of wind',
 'V component of wind',
 'Relative humidity',
 'Surface pressure',
 'Volumetric soil moisture content',
 'Water equivalent of accumulated snow depth (deprecated)',
 'Tropopause pressure',
 'Pressure',
 'Land-sea mask',
 'Sea ice area fraction']

In [53]:
messages[2015] = [msg for msg in messages[2015] if msg['name'] in info]

In [57]:
messages[2014][0].keys()

['globalDomain',
 'GRIBEditionNumber',
 'tablesVersionLatestOfficial',
 'tablesVersionLatest',
 'grib2divider',
 'angleSubdivisions',
 'missingValue',
 'ieeeFloats',
 'isHindcast',
 'section0Length',
 'identifier',
 'discipline',
 'editionNumber',
 'totalLength',
 'sectionNumber',
 'section1Length',
 'numberOfSection',
 'centre',
 'centreDescription',
 'subCentre',
 'tablesVersion',
 'masterDir',
 'localTablesVersion',
 'significanceOfReferenceTime',
 'year',
 'month',
 'day',
 'hour',
 'minute',
 'second',
 'dataDate',
 'julianDay',
 'dataTime',
 'productionStatusOfProcessedData',
 'typeOfProcessedData',
 'md5Section1',
 'selectStepTemplateInterval',
 'selectStepTemplateInstant',
 'stepType',
 'is_chemical',
 'is_chemical_distfn',
 'is_chemical_srcsink',
 'is_aerosol',
 'is_aerosol_optical',
 'setCalendarId',
 'deleteCalendarId',
 'sectionNumber',
 'grib2LocalSectionPresent',
 'deleteLocalDefinition',
 'sectionNumber',
 'gridDescriptionSectionPresent',
 'section3Length',
 'numberOfSec

In [63]:
messages[2014][0]['name']

'Mean sea level pressure'

In [54]:
messages[2015]

[1:U component of wind:m s**-1 (instant):regular_ll:planetaryBoundaryLayer:level 0:fcst time 0 hrs:from 201506160000,
 2:V component of wind:m s**-1 (instant):regular_ll:planetaryBoundaryLayer:level 0:fcst time 0 hrs:from 201506160000,
 5:Geopotential height:gpm (instant):regular_ll:isobaricInhPa:level 1000 Pa:fcst time 0 hrs:from 201506160000,
 6:Temperature:K (instant):regular_ll:isobaricInhPa:level 1000 Pa:fcst time 0 hrs:from 201506160000,
 7:Relative humidity:% (instant):regular_ll:isobaricInhPa:level 1000 Pa:fcst time 0 hrs:from 201506160000,
 8:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 1000 Pa:fcst time 0 hrs:from 201506160000,
 9:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 1000 Pa:fcst time 0 hrs:from 201506160000,
 12:Geopotential height:gpm (instant):regular_ll:isobaricInhPa:level 2000 Pa:fcst time 0 hrs:from 201506160000,
 13:Temperature:K (instant):regular_ll:isobaricInhPa:level 2000 Pa:fcst time 0 hrs:from 201506160000,
 